<a href="https://colab.research.google.com/github/RohithOfRivia/SMILES-Toxicity-Prediction/blob/main/19/02_F1-52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
!pip install rdkit
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (GradientBoostingClassifier, 
                              HistGradientBoostingClassifier)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import resample


from sklearn.metrics import f1_score, recall_score

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from xgboost import XGBClassifier
import time
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
train_data_url = "https://raw.githubusercontent.com/RohithOfRivia/SMILES-Toxicity-Prediction/main/Data/train_II.csv"
test_data_url = "https://raw.githubusercontent.com/RohithOfRivia/SMILES-Toxicity-Prediction/main/Data/test_II.csv"

df = pd.read_csv(train_data_url)

In [32]:
#transforming each compound into their canonical SMILES format. Optional.
def canonicalSmiles(smile):
    try:
        return Chem.MolToSmiles(Chem.MolFromSmiles(smile))
    except:
        return(Chem.MolToSmiles(Chem.MolFromSmiles("[Na+].[Na+].F[Si--](F)(F)(F)(F)F")))

In [122]:
#Read data and split up the given features
class FileReadTransform(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    #training and test data are slightly different, hence passing optional test param
    def transform(self, X, test=False):
        
        try:
          # if test == False:
            X['SMILES'] = X['Id'].apply(lambda x: x.split(';')[0])
            X['assay'] = X['Id'].apply(lambda x: x.split(';')[1])
        
        except KeyError:
            X['SMILES'] = X['x'].apply(lambda x: x.split(';')[0])
            X['assay'] = X['x'].apply(lambda x: x.split(';')[1])
          
        print("FileReadTransform done")
        
        #correct smiles for this compound found through https://www.molport.com/shop/index
        #X["SMILES"] = X["SMILES"].replace({"F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]":"[Na+].[Na+].F[Si--](F)(F)(F)(F)F"})
        
        #Deleting invalid compound from the data
        X = X.loc[X.SMILES != "F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]"]
        return X
    
    
class CanonicalGenerator(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['SMILES'] = X['SMILES'].apply(canonicalSmiles)
        print("CanonicalGenerator done")
        return X

    
class FingerprintGenerator(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
          #tracks each unique compound and its fingerprints
          tracker = []
          fps = []
          assays = []
          unique = len(X['SMILES'].unique())
          counter = 0

          for index, columns in X[["SMILES", "assay"]].iterrows():

              #skip if already in tracker
              if columns[0] in tracker:
                  continue

              #append each unique compound and thier respective fingerprints
              else:
                  tracker.append(columns[0])
                  assays.append(columns[1])

                  mol = Chem.MolFromSmiles(columns[0])
                  fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
                  fps.append(fp.ToList())

                  counter += 1

                  # print(f"compound {counter}/{unique}...

          #Combining all compounds, assays and fingerprints into one dataframe 
          cols = a = ["x" + str(i) for i in range (1, 1025)]
          smiles_df = pd.DataFrame(columns=['SMILES'], data=tracker)
          fingerprints = pd.DataFrame(columns=cols, data=fps)

          df = pd.concat([smiles_df, fingerprints], axis=1)

          print("FingerprintGenerator done")
          return pd.merge(X, df, on='SMILES') 


#Feature reduction with variance threshold 
class VarianceThresh(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, thresh=.8):
      
      #Looks to columns to determine whether X is training or testing data 
      cols = X.columns
      if 'x' in cols:
        temp_df = X.drop(columns=["x", "assay", "SMILES"])
        cols = ["x", "assay", "SMILES"]
      else:
        temp_df = X.drop(columns=["Id", "Expected","assay", "SMILES"])
        cols = ["Id", "Expected","assay", "SMILES"]

      #Selecting features based on the variance threshold
      selector = VarianceThreshold(threshold=(thresh * (1 - thresh))) 
      selector.fit(temp_df)

      #This line transforms the data while keeping the column names 
      temp_df = temp_df.loc[:, selector.get_support()]

      #Attaching the ids, assays, smiles etc. that is still required for model
      return pd.concat([X[cols], temp_df], axis=1) , selection

      

## Generating descriptors

In [34]:
class DescriptorGenerator(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
        return self
    
  def transform(self, X):
    #Initializing descriptor calculator
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    #Tracking each unique compound and generating descriptors 
    tracker = []
    descriptors = []
    for compound in X['SMILES']:

      if compound in tracker:
        continue

      else:
        tracker.append(compound)

        mol = Chem.MolFromSmiles(compound)
        current_descriptors = calc.CalcDescriptors(mol)
        descriptors.append(current_descriptors)

    # Combining X, SMILES, and generated descriptors 
    df = pd.DataFrame(descriptors,columns=desc_names)
    temp_df = pd.DataFrame(tracker, columns=["SMILES"])
    df = pd.concat([df, temp_df], axis=0)

    print("DescriptorGenerator done")
    return pd.merge(X, df, on='SMILES')

In [125]:
feature_generation_pipeline = Pipeline(steps=[
    ('read', FileReadTransform()),
     ('canon', CanonicalGenerator()),
     ('fpr', FingerprintGenerator()),
     ('desc', DescriptorGenerator())
     ])

feature_selector_pipeline = Pipeline(steps=[
    ('vtr', VarianceThresh()),
     ])
df_processed = feature_generation_pipeline.fit_transform(df)
df_processed, selector = feature_selector_pipeline.fit_transform(df_processed)

FileReadTransform done
CanonicalGenerator done
FingerprintGenerator done
DescriptorGenerator done


In [126]:
df_processed

,Id,Expected,assay,SMILES,x2,x34,x65,x81,x115,x129,...,fr_benzene,fr_bicyclic,fr_ester,fr_ether,fr_halogen,fr_methoxy,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_unbrch_alkane
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,2,1644,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,0,0,0,1,0,...,2,0,0,0,3,0,0,2,2,0
1,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1630,2,1630,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,0,0,0,1,0,...,2,0,0,0,3,0,0,2,2,0
2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;29,2,29,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,0,0,0,1,0,...,2,0,0,0,3,0,0,2,2,0
3,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1618,2,1618,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,0,0,0,1,0,...,2,0,0,0,3,0,0,2,2,0
4,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1638,2,1638,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,0,0,0,1,0,...,2,0,0,0,3,0,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,C1=CC=C(C=C1)NC(=S)N;1852,1,1852,NC(=S)Nc1ccccc1,0,0,1,0,0,1,...,1,0,0,0,0,0,1,0,0,0
75373,CN1CN(CN(C1)C)C;2,2,2,CN1CN(C)CN(C)C1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75374,CCCCC1CCC(=O)O1;1852,1,1852,CCCCC1CCC(=O)O1,0,1,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
75375,CCOC(=O)CCC1=CC=CC=C1;2,2,2,CCOC(=O)CCc1ccccc1,0,1,1,1,0,0,...,1,0,1,1,0,0,0,0,0,0


In [143]:
# model = HistGradientBoostingClassifier()
# # define the evaluation procedure
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate the model and collect the scores
# X = df_processed.drop(columns=['Id', 'Expected', "SMILES"])
# n_scores = cross_val_score(model, X, df_processed['Expected'], scoring="f1", cv=cv, n_jobs=-1)
# # report performance
# print('F1: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

F1: 0.521 (0.014)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(df_fp.drop(columns=['Expected', "Id", "SMILES"]), df_fp["Expected"].values, test_size=0.3, stratify=df_fp['Expected'],random_state=5)

# # RFmodel = RandomForestClassifier(n_estimators=500, random_state=1, max_depth=7, )
# # RFmodel.fit(X_train, y_train)
# # y_predRF = RFmodel.predict(X_test)

# KNN = KNeighborsClassifier()
# KNN.fit(X_train, y_train)
# y_predRF = KNN.predict(X_test)

# # from sklearn.linear_model import SGDClassifier
# # lr = SGDClassifier(loss='perceptron', random_state=5)
# # lr.fit(X_train, y_train)

# # lr_params = {'alpha' : [10**(-x) for x in range(7)],
# #              'penalty' : ['l1', 'l2', 'elasticnet'],
# #              'l1_ratio' : [0.15, 0.25, 0.5, 0.75]}

# # y_predRF = lr.predict(X_train)

# # Print the accuracy score
# print(accuracy_score(y_test, y_predRF))
# print(f1_score(y_test, y_predRF, zero_division=0))

In [ ]:
# # Id = X['x'].values

# #Switch off the line below when using training data
# X = df_fp.drop(["Expected", "Id", "SMILES"], axis=1)

# #Switch off the line below when using testing data
# # X = df_fp.drop(["x", "SMILES"], axis=1)

# # Initialize the scaler
# scaler = StandardScaler()

# # Fit the scaler to the data
# scaler.fit(X)

# # Scale the data
# X_scaled = scaler.transform(X)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, df_fp["Expected"].values, test_size=0.3, stratify=df_fp['Expected'],random_state=5)

# # RFmodel = RandomForestClassifier(n_estimators=500, random_state=1, max_depth=7, )
# # RFmodel.fit(X_train, y_train)
# # y_predRF = RFmodel.predict(X_test)

# hgbm = HistGradientBoostingClassifier(random_state=42)
# hgbm.fit(X_train, y_train)
# hgbm.score(X_test, y_test)


# print(accuracy_score(y_test, y_predRF))
# print(f1_score(y_test, y_predRF, zero_division=0))

In [ ]:
# y_predRF = hgbm.predict(X_test)
# print(f1_score(y_test, y_predRF, zero_division=0))